In [1]:
!pip install streamlit
!pip install streamlit_ace

In [2]:
%%writefile bickroy_recommender.py
import pickle
import os
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

current_dir = os.path.dirname(__file__)
count_matrix=pickle.load(open(current_dir + '/bickroy.pkl','rb'))
cosine_sim = cosine_similarity(count_matrix, count_matrix)
dataset = pd.read_csv(current_dir + '/bickroy.csv')
dataset = dataset.set_index('title')
indices = pd.Series(dataset.index)

def bickroy_recommendations(title):
    recommended_gadgets = []
    matches = indices[indices == title]
    if matches.shape[0] == 0:
        f = indices.apply(lambda x: " ".join(x.split()))
        matches = f[f.str.contains(title)]
    if matches.shape[0] == 0:
        return []
    idx = matches.index[0]
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending=False)
    top_indexes = list(score_series.iloc[1:].index)
    items = list(dataset.index)
    
    for i in top_indexes:
        item = items[i]
        if item != title:
            recommended_gadgets.append(items[i])
        
    return list(set(recommended_gadgets))[:10]

Overwriting bickroy_recommender.py


In [3]:
%%writefile rokomari_recommender.py
import pickle
import os
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

current_dir = os.path.dirname(__file__)
count_matrix=pickle.load(open(current_dir + '/count_matrix.pkl','rb'))
cosine_sim = cosine_similarity(count_matrix, count_matrix)
df2 = pd.read_csv(current_dir + '/df2.csv')
df2 = df2.set_index('Title')
indices = pd.Series(df2.index)

def rokomari_recommendations(title):
    recommended_books = []
    recommended_books_wa = []
    matches = indices[indices == title]
    if matches.shape[0] == 0:
        f = indices.apply(lambda x: " ".join(x.split()))
        matches = f[f.str.contains(title)]
    if matches.shape[0] == 0:
        return []
    idx = matches.index[0]
    
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending=False)
    top_10_indexes = list(score_series.iloc[1:51].index)
    
    for i in top_10_indexes:
        recommended_books.append(list(df2.index)[i])
    for i in top_10_indexes:
        recommended_books_wa.append(list(df2.weighted_average)[i])
    Book_list_poplarity = pd.DataFrame(
        { 'Title':recommended_books,
          'Popularity': recommended_books_wa
        })
        
    return Book_list_poplarity.sort_values('Popularity',ascending=False)


Overwriting rokomari_recommender.py


In [4]:
%%writefile floralimited_recommender.py
import pickle
import os
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

current_dir = os.path.dirname(__file__)
tfidf_matrix=pickle.load(open(current_dir + '/floralimited.pkl','rb'))
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
df = pd.read_csv(current_dir + '/floralimited.csv')
df = df.set_index('name')
indices = pd.Series(df.index)

def flora_recommender(title):
    recommended_products = []
    matches = indices[indices == title]
    if matches.shape[0] == 0:
        f = indices.apply(lambda x: " ".join(x.split()))
        matches = f[f.str.contains(title)]
    if matches.shape[0] == 0:
        return []
    idx = matches.index[0]
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending=False)
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    for i in top_10_indexes:
        recommended_products.append(list(df.index)[i])
    return recommended_products

Overwriting floralimited_recommender.py


In [5]:
%%writefile configurations.py
from floralimited_recommender import flora_recommender
from rokomari_recommender import rokomari_recommendations
from bickroy_recommender import bickroy_recommendations

bickroy_recommendations
models = [
    {
        'name': 'Floralimited Recommender', 
        'handler': flora_recommender,
    },
    {
        'name': 'Rokomari Recommender', 
        'handler': rokomari_recommendations,
    },
    {
        'name': 'Bickroy Recommender', 
        'handler': bickroy_recommendations,
    }
]
style_placeholders = {
    '__primary_color__': 'black',
    '__secondary_color__': '#ddd'
}

Overwriting configurations.py


In [6]:
%%writefile app.py

import streamlit as st
import os
import configurations as config
from SessionState import get

def css():
    with open(os.path.dirname(__file__) + "/style.css") as file:
        content = file.read()
        #file style attrs
        for key, value in config.style_placeholders.items():
            content = content.replace(key, value)
        st.markdown(content, unsafe_allow_html = True)
state = get(button_pressed = None)
def get_state_handler(model):
    def state_handler():
        state.button_pressed = model['name']
        return model['name']
    return state_handler
def main():
    css()
    #render the models
    for index, model in enumerate(config.models):
        b = st.sidebar.button(model['name'], on_click=get_state_handler(model))
        if b or (~b and state.button_pressed == model['name']):
            title = st.text_input('Product Title:')
            if st.button('Recommend'):
                recommendations = model['handler'](title)
                st.write(recommendations)
if __name__=='__main__':
    main()

Overwriting app.py
